In [ ]:
# Colab compatitbility
from google.colab import drive
drive.mount('/content/drive')
 
%cd /content/drive/My\ Drive/Colab\ Notebooks/URECA/GNNExplainer

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/URECA/GNNExplainer


In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 8.3MB/s 


In [ ]:
import argparse
import os

import sklearn.metrics as metrics

from tensorboardX import SummaryWriter

import pickle
import shutil
import torch

import models
import utils.io_utils as io_utils
import utils.parser_utils as parser_utils
from explainer import explain

import gengraph
from utils import featgen
import numpy as np

In [ ]:
class Args():
  pass
argObj = Args()
argObj.gpu = False
argObj.method = True
argObj.bias = True

In [ ]:
writer = None

# Load a model checkpoint
ckpt = torch.load("ckpt/syn1_base_h20_o20.pth.tar")
cg_dict = ckpt["cg"] # get computation graph
input_dim = cg_dict["feat"].shape[2] 
num_classes = cg_dict["pred"].shape[2]

# build model
model = models.GcnEncoderNode(
    input_dim=input_dim,
    hidden_dim=20,
    embedding_dim=20,
    label_dim=num_classes,
    num_layers=3,
    bn=False,
    args=argObj,
)

# load state_dict (obtained by model.state_dict() when saving checkpoint)
model.load_state_dict(ckpt["model_state"])

G, labels, name = gengraph.gen_syn1(
        feature_generator=featgen.ConstFeatureGen(np.ones(input_dim, dtype=float)))
num_classes = max(labels) + 1

data = gengraph.preprocess_input_graph(G, labels)

adj_ = torch.tensor(data["adj"], dtype=torch.float)
x_ = torch.tensor(data["feat"], dtype=torch.float)

Using CPU
feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32


In [ ]:
model.zero_grad()
# not sure why adj_ varies from adj
output_ = model(x_, adj_)

adj = torch.tensor(cg_dict["adj"], dtype=torch.float)
x = torch.tensor(cg_dict["feat"], dtype=torch.float)
output = model(x, adj)

In [ ]:
metrics.accuracy_score(labels, output[0][0].argmax(1))

0.9885714285714285

In [ ]:
metrics.accuracy_score(labels, output_[0][0].argmax(1))

0.9785714285714285

In [ ]:
full_pred = output[0][0].argmax(1)

# With subgraph



In [ ]:
import utils.graph_utils as graph_utils

Load masks

In [ ]:
logdir = './log/'
expdir = 'syn1_base_h20_o20_explain'

dirs = os.listdir(os.path.join(logdir, expdir))

masks = []
for file in dirs:
    if file.split('.')[-1] == 'npy':
        print(file)
        masks.append(file)

In [ ]:
# adj = torch.tensor(data["adj"], dtype=torch.float)
adj = cg_dict["adj"]
feat = cg_dict["feat"]

neighborhoods = graph_utils.neighborhoods(adj=adj, n_hops=3, use_cuda=True)

def extract_neighborhood(node_idx, graph_idx=0):
    """Returns the neighborhood of a given node."""
    neighbors_adj_row = neighborhoods[graph_idx][node_idx, :]
    # index of the query node in the new adj
    node_idx_new = sum(neighbors_adj_row[:node_idx])
    neighbors = np.nonzero(neighbors_adj_row)[0]
    sub_adj = adj[graph_idx][neighbors][:, neighbors]
    sub_feat = feat[graph_idx, neighbors]
    sub_label = labels[neighbors]
    return node_idx_new, sub_adj, sub_feat, sub_label, neighbors

def filter_adj(adj):
    filt_adj = adj.copy()
    filt_adj[adj>0.8] = -1
    return filt_adj

np.random.seed(0)

# apply mask
for m in masks: 
    adj_mask = np.load(os.path.join(logdir, expdir, m))
    node_idx = int(m.split("_idx_")[1].split("graph")[0])
    node_idx_new, sub_adj, sub_feat, sub_label, neighbors = extract_neighborhood(node_idx)
    
    sub_feat = torch.tensor([sub_feat], dtype=torch.float)
    sub_adj = torch.tensor([sub_adj], dtype=torch.float)
    adj_mask = torch.tensor([adj_mask], dtype=torch.float)

    full_pred = model(sub_feat, sub_adj)
    full_pred = full_pred[0][0].argmax(1)

    GNN_pred = model(sub_feat, sub_adj - adj_mask)
    GNN_pred = GNN_pred[0][0].argmax(1)

    im_links_to_remove = adj_mask.argmax(1)
    im_sub_adj = sub_adj.clone()
    im_adj_mask = sub_adj.clone()
    i = 0
    for l in im_links_to_remove:
        im_sub_adj[0][l][i] = 0
        im_adj_mask[0][i][l] = 0
        im_sub_adj[0][l][i] = 0
        im_adj_mask[0][i][l] = 0
        i += 1
    im_GNN_pred = model(sub_feat, im_sub_adj - im_adj_mask)
    im_GNN_pred = im_GNN_pred[0][0].argmax(1)
    
    print("Node index: %d" %(node_idx))
    n = len(neighbors)
    acc1 = metrics.accuracy_score(sub_label, full_pred, normalize=False)
    acc2 = metrics.accuracy_score(sub_label, GNN_pred, normalize=False)
    acc3 = metrics.accuracy_score(sub_label, im_GNN_pred, normalize=False)
    print("Without GNN: %d / %d (%f)" %(acc1, n, acc1/n))
    print("With GNN: %d / %d (%f)" %(acc2, n, acc2/n))
    print("With GNN, missing important links: %d / %d (%f)" %(acc3, n, acc3/n))
    print("")

Node index: 400
Without GNN: 104 / 109 (0.954128)
With GNN: 100 / 109 (0.917431)
With GNN, missing important links: 2 / 109 (0.018349)

Node index: 405
Without GNN: 112 / 126 (0.888889)
With GNN: 108 / 126 (0.857143)
With GNN, missing important links: 2 / 126 (0.015873)

Node index: 410
Without GNN: 92 / 112 (0.821429)
With GNN: 89 / 112 (0.794643)
With GNN, missing important links: 4 / 112 (0.035714)

Node index: 415
Without GNN: 94 / 111 (0.846847)
With GNN: 91 / 111 (0.819820)
With GNN, missing important links: 4 / 111 (0.036036)

Node index: 420
Without GNN: 117 / 128 (0.914062)
With GNN: 114 / 128 (0.890625)
With GNN, missing important links: 2 / 128 (0.015625)

Node index: 425
Without GNN: 151 / 164 (0.920732)
With GNN: 148 / 164 (0.902439)
With GNN, missing important links: 2 / 164 (0.012195)

Node index: 430
Without GNN: 178 / 188 (0.946809)
With GNN: 174 / 188 (0.925532)
With GNN, missing important links: 2 / 188 (0.010638)

Node index: 435
Without GNN: 117 / 132 (0.886364)
Wi

In [ ]:
adj_mask.argmax(1)

tensor([[ 79, 157,  36,   5,  49, 134,  32,  14,  60,  11, 157,   9, 143, 109,
          92, 157,   6,  40,  88,   9,  12,  60,  57,   3, 151,  17,  99,  34,
          17,   4,  34,   2,   6,   6, 134,  13,   2,   6, 125,  34,  17,  93,
           8, 151,  11,   2,   6,  11, 132, 132,  26, 140, 126,  10,  76,   1,
           4, 132, 134,   6,  14, 134,  61,  69,  58,   8, 157, 138,  58,  63,
         122,  34,  11,   6,   5,   4,  54,   3, 151,   0,  64,  31,  11, 151,
           5,  18,   2,  14,  18, 117, 119,  93,  14,  41,   5,  34,  60,   9,
          87,  26,   5,  84,   6,  58,   5,   5,  30,  66, 157,  13,  61,   4,
          32,  61,  79, 121,  94,  89,  57,  90,  10,  34,  84,  33,  71,  38,
          52,  69,   8,  61,  58, 124, 134,   5, 161,  10,  30,  89,  67,   6,
          61,   5, 155,  12,   1, 147,  10, 145,  34,   1,   5, 134,  20,   5,
          10,  58,  22, 161,   5,  61, 108, 162, 161, 162, 163, 162]])

# Reference

In [ ]:
dir(models.GcnEncoderNode)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_name',
 '_load_from_state_dict',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'apply_bn',
 'bfloat16',
 'buffers',
 'build_conv_layers',
 'build_pred_layers',
 'children',
 'construct_mask',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'gcn_forward',
 'half',
 'load_state_dict',
 'loss',
 'modules',
 'named_